In [23]:
from gurobipy import Model, multidict, GRB
import pandas as pd

df = pd.read_excel("data.xlsx")
# Define the list of DMUs
# DMU = ["A", "B", "C", "D", "E"]

# Create dictionaries to store various information
E = {}
val_p1, val_p2, val_p3, val_s1, val_s2 = {}, {}, {}, {}, {}
slack_p1, slack_p2, slack_p3 = {}, {}, {}

# Define the number of inputs (I) and outputs (O) for each DMU
I = 3
O = 3

In [24]:
# Define data specific to each DMU
# DMU, Totx1, Totx2, Totx3 = multidict(
#     {
#         "A": [11, 14, 60],
#         "B": [7, 7, 60],
#         "C": [11, 14, 60],
#         "D": [14, 14, 60],
#         "E": [14, 15, 60],
#     }
# )
Tot = df[df["process"]=="total"][["DMU","x1","x2","x3"]].set_index("DMU")
DMU = Tot.index.to_list()
Tot=Tot.to_dict()

# Define more data specific to each DMU (process 1)
# DMU, proc1x1, proc1x2, proc1x3, proc1TotyO, proc1yO, proc1yI = multidict(
#     {
#         "A": [3, 5, 10, 4, 2, 2],
#         "B": [2, 3, 10, 2, 1, 1],
#         "C": [3, 4, 10, 2, 1, 1],
#         "D": [4, 6, 10, 3, 2, 1],
#         "E": [5, 6, 10, 4, 3, 1],
#     }
# )
proc1 = df[df["process"]==1][["DMU","x1","x2","x3","y1-O","y1-I"]].set_index("DMU")
proc1.rename(columns={"y1-O":"yO","y1-I":"yI"},inplace=True)
proc1["TotyO"] = proc1["yO"]+proc1["yI"]
proc1 = proc1.to_dict()

# Define more data specific to each DMU (process 2)
# DMU, proc2x1, proc2x2, proc2x3, proc2TotyO, proc2yO, proc2yI = multidict(
#     {
#         "A": [4, 3, 20, 3, 2, 1],
#         "B": [2, 1, 20, 2, 1, 1],
#         "C": [5, 3, 20, 2, 1, 1],
#         "D": [5, 5, 20, 4, 3, 1],
#         "E": [5, 4, 20, 4, 2, 2],
#     }
# )
proc2 = df[df["process"]==2][["DMU","x1","x2","x3","y2-O","y2-I"]].set_index("DMU")
proc2.rename(columns={"y2-O":"yO","y2-I":"yI"},inplace=True)
proc2["TotyO"] = proc2["yO"]+proc2["yI"]
proc2 = proc2.to_dict()

# Define more data specific to each DMU (process 3)
# DMU, proc3x1, proc3x2, proc3x3, proc3TotyO = multidict(
#     {
#         "A": [4, 6, 30, 1],
#         "B": [3, 3, 30, 1],
#         "C": [3, 7, 30, 2],
#         "D": [5, 3, 30, 1],
#         "E": [4, 5, 30, 3],
#     }
# )
proc3 = df[df["process"]==3][["DMU","x1","x2","x3","y3"]].set_index("DMU")
proc3.rename(columns={"y3":"TotyO"},inplace=True)
# proc3["TotyO"] = proc3["yO"]+proc3["yI"]
proc3 = proc3.to_dict()

P1, P2, P3 = {}, {}, {}
v, u = {}, {}

In [25]:
for k in DMU:
    # Create the Gurobi model
    m = Model("network_DEA")

    # Define decision variables
    v = {i: m.addVar(vtype=GRB.CONTINUOUS, name=f"v_{i}") for i in range(I)}
    u = {i: m.addVar(vtype=GRB.CONTINUOUS, name=f"u_{i}") for i in range(O)}

    # Update the model
    m.update()

    # Set the objective function to maximize efficiency
    m.setObjective(
        u[0] * proc1["yO"][k] + u[1] * proc2["yO"][k] + u[2] * proc3["TotyO"][k],
        GRB.MAXIMIZE,
    )

    # Add constraints
    m.addConstr(v[0] * Tot["x1"][k] + v[1] * Tot["x2"][k] + v[2] * Tot["x3"][k] == 1)

    for j in DMU:
        P1[j] = m.addConstr(
            u[0] * proc1["TotyO"][j]
            - (v[0] * proc1["x1"][j] + v[1] * proc1["x2"][j] + v[2] * proc1["x3"][j])
            <= 0
        )
        P2[j] = m.addConstr(
            u[1] * proc2["TotyO"][j]
            - (v[0] * proc2["x1"][j] + v[1] * proc2["x2"][j] + v[2] * proc2["x3"][j])
            <= 0
        )
        P3[j] = m.addConstr(
            u[2] * proc3["TotyO"][j]
            - (
                v[0] * proc3["x1"][j]
                + v[1] * proc3["x2"][j]
                + v[2] * proc3["x3"][j]
                + u[0] * proc1["yI"][j]
                + u[1] * proc2["yI"][j]
            )
            <= 0
        )

    # Optimize the model
    m.optimize()

    # Store efficiency values in the dictionary
    # E[k] = f"The efficiency of DMU {k}: {m.objVal}"
    E[k] = m.objVal

    u_sol = m.getAttr("x", u)
    v_sol = m.getAttr("x", v)

    # Calculate process and stage efficiencies
    E1 = (
        u_sol[0]
        * proc1["TotyO"][k]
        / (
            v_sol[0] * proc1["x1"][k]
            + v_sol[1] * proc1["x2"][k]
            + v_sol[2] * proc1["x3"][k]
        )
    )
    E2 = (
        u_sol[1]
        * proc2["TotyO"][k]
        / (
            v_sol[0] * proc2["x1"][k]
            + v_sol[1] * proc2["x2"][k]
            + v_sol[2] * proc2["x3"][k]
        )
    )
    E3 = (
        u_sol[2]
        * proc3["TotyO"][k]
        / (
            v_sol[0] * proc3["x1"][k]
            + v_sol[1] * proc3["x2"][k]
            + v_sol[2] * proc3["x3"][k]
            + u_sol[0] * proc1["yI"][k]
            + u_sol[1] * proc2["yI"][k]
        )
    )
    stage1 = (
        u_sol[0] * proc1["TotyO"][k]
        + u_sol[1] * proc2["TotyO"][k]
        + v_sol[0] * proc3["x1"][k]
        + v_sol[1] * proc3["x2"][k]
        + v_sol[2] * proc3["x3"][k]
    ) / (v_sol[0] * Tot["x1"][k] + v_sol[1] * Tot["x2"][k] + v_sol[2] * Tot["x3"][k])
    stage2 = (
        u_sol[0] * proc1["yO"][k]
        + u_sol[1] * proc2["yO"][k]
        + u_sol[2] * proc3["TotyO"][k]
    ) / (
        u_sol[0] * proc1["TotyO"][k]
        + u_sol[1] * proc2["TotyO"][k]
        + v_sol[0] * proc3["x1"][k]
        + v_sol[1] * proc3["x2"][k]
        + v_sol[2] * proc3["x3"][k]
    )

    # Store process and stage efficiency values
    val_p1[k] = f"The efficiency of process 1 of DMU {k}: {E1:.4g}"
    val_p2[k] = f"The efficiency of process 2 of DMU {k}: {E2:.4g}"
    val_p3[k] = f"The efficiency of process 3 of DMU {k}: {E3:.4g}"
    val_s1[k] = f"The efficiency of stage 1 of DMU {k}: {stage1:.4g}"
    val_s2[k] = f"The efficiency of stage 2 of DMU {k}: {stage2:.4g}"

    # Calculate and store inefficiency values
    process1_slack = m.getAttr("slack", P1)
    slack_p1[k] = f"The inefficiency of process 1 of DMU {k}: {process1_slack[k]:.4g}"
    process2_slack = m.getAttr("slack", P2)
    slack_p2[k] = f"The inefficiency of process 2 of DMU {k}: {process2_slack[k]:.4g}"
    process3_slack = m.getAttr("slack", P3)
    slack_p3[k] = f"The inefficiency of process 3 of DMU {k}: {process3_slack[k]:.4g}"

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 6 columns and 73 nonzeros
Model fingerprint: 0x14dc8595
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 16 rows, 6 columns, 73 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5625000e+30   8.921875e+30   4.562500e+00      0s
       5    3.9726940e-01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.972693973e-01
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 6 columns and 73 nonzeros
Model fingerprint: 0xf9c8

In [26]:
# Display the results for each DMU
for k in DMU:
    print(E[k])
    print(val_p1[k])
    print(val_p2[k])
    print(val_p3[k])
    print(val_s1[k])
    print(val_s2[k])
    print(slack_p1[k])
    print(slack_p2[k])
    print(slack_p3[k])
    print("--------------")

0.39726939726939725
The efficiency of process 1 of DMU A: 0.01732
The efficiency of process 2 of DMU A: 0.75
The efficiency of process 3 of DMU A: 0.8879
The efficiency of stage 1 of DMU A: 0.4253
The efficiency of stage 2 of DMU A: 0.934
The inefficiency of process 1 of DMU A: 0.4913
The inefficiency of process 2 of DMU A: 0.08333
The inefficiency of process 3 of DMU A: 0.02806
--------------
0.7854288568574284
The efficiency of process 1 of DMU B: 1
The efficiency of process 2 of DMU B: 1
The efficiency of process 3 of DMU B: 0.5712
The efficiency of stage 1 of DMU B: 1
The efficiency of stage 2 of DMU B: 0.7854
The inefficiency of process 1 of DMU B: 0
The inefficiency of process 2 of DMU B: 0
The inefficiency of process 3 of DMU B: 0.2146
--------------
0.5611888111888111
The efficiency of process 1 of DMU C: 0.2008
The efficiency of process 2 of DMU C: 0.5
The efficiency of process 3 of DMU C: 0.7619
The efficiency of stage 1 of DMU C: 0.7001
The efficiency of stage 2 of DMU C: 0.

In [32]:
dict(sorted(E.items(), key=lambda item: item[1],reverse=True))

{'B': 0.7854288568574284,
 'C': 0.5611888111888111,
 'D': 0.4964202464202464,
 'E': 0.4941724941724942,
 'A': 0.39726939726939725}